In [1]:
import datetime
import numpy as np
import pandas as pd

from tqdm import tqdm, trange

from airsim.collections import AirObject, AirEnv, RadarSystem, ControlPoint, Supervisor
from airsim.time import Time

## Конфигурация

In [2]:
cp_data_dir = 'CP_data'
data_usage_aim = 'train'

## Генерация

In [3]:
def get_random_air_object(xyz_mean, xyz_std, v_mean, v_std, a_mean, a_std):
    x_0 = xyz_mean[0] + np.random.randn() * xyz_std[0]
    y_0 = xyz_mean[1] + np.random.randn() * xyz_std[1]
    z_0 = xyz_mean[2] + np.random.randn() * xyz_std[2]
    
    v_x = (v_mean + np.random.randn() * v_std) * np.random.choice([1, -1])
    v_y = (v_mean + np.random.randn() * v_std) * np.random.choice([1, -1])
    v_z = (v_mean + np.random.randn() * v_std) * np.random.choice([1, -1])
    
    a_x = (a_mean + np.random.randn() * a_std) * np.random.choice([1, -1])
    a_y = (a_mean + np.random.randn() * a_std) * np.random.choice([1, -1])
    a_z = (a_mean + np.random.randn() * a_std) * np.random.choice([1, -1])

    f = lambda t: np.array([x_0 + v_x*t + a_x*t**2/2, y_0 + v_z*t + a_z*t**2/2, z_0 + v_z*t + a_z*t**2/2])

    return AirObject(track=f)

In [4]:
def get_radar_systems_grid(size, distance, dt):
    n = size**2
    radar_systems = []

    for i in range(n):
        detection_period = np.random.choice([100, 500, 1000, 5000], p=[0.5, 0.3, 0.1, 0.1])
        detection_delay = np.random.randint(0, detection_period//dt)*dt
        radar_systems.append(
            RadarSystem(position=np.array([distance * (i // size) - (size - 1) / 2 * distance, distance * (i % size) - (size - 1) / 2 * distance, 0.0]),
                        detection_radius=np.sqrt(10000**2 + (0.8*distance)**2),
                        error=np.random.choice([1.0, 2.5, 5.0, 10.0], p=[0.1, 0.5, 0.3, 0.1]),
                        detection_fault_probability=np.random.choice([0.01, 0.02, 0.05, 0.2], p=[0.2, 0.5, 0.2, 0.1]),
                        detection_period=detection_period,
                        detection_delay=detection_delay)
        )

    return radar_systems 

In [5]:
n_ao = 50
rs_grid_size = 5

In [6]:
t_min = 0
t_max = 300 * 1000
dt = 10

In [7]:
air_objects = [
    get_random_air_object(xyz_mean=(0.0, 0.0, 10000.0),
                          xyz_std=(50000.0, 50000.0, 100.0),
                          v_mean=0.3, v_std=0.2,
                          a_mean=1e-5, a_std=1e-6)
    for _ in range(50)
]

In [8]:
air_env = AirEnv(air_objects=air_objects)

In [9]:
radar_systems = get_radar_systems_grid(rs_grid_size, 20000, dt)

In [10]:
supervisor = Supervisor(air_env=air_env, radar_systems=radar_systems)

In [11]:
supervisor.run(t_min, t_max, dt)

Running system: 100%|█████████████████████████████████████████| 30001/30001 [15:43<00:00, 31.78it/s]


In [12]:
cp_data = supervisor.get_data()

In [13]:
cp_data.groupby(by=['rs_id', 'id'])['time'].describe()[['count', 'min', 'max']]

count      min      max
rs_id id                         
0     18   58.0   4340.0  33840.0
      20   58.0    340.0  29840.0
      40    9.0    340.0   4340.0
1     18    5.0   4740.0  24740.0
      20    6.0   9740.0  34740.0
...         ...      ...      ...
24    16  151.0    470.0  75970.0
      26   45.0    470.0  22470.0
      30   19.0  31970.0  40970.0
      42  104.0    470.0  52470.0
      47   23.0    470.0  11470.0

[79 rows x 3 columns]

In [14]:
cp_data.to_csv(f'{cp_data_dir}/{data_usage_aim}/{n_ao}ao_{rs_grid_size**2}rs_xyz_quadratic.csv')